In [ ]:
!pip3 install torch --extra-index-url https://download.pytorch.org/whl/cu116
!pip3 install rdkit
!pip3 install git+https://github.com/molecularsets/moses.git

In [ ]:
import torch
import pandas as pd
import os
import sys
import time
from datetime import timedelta
from importlib import reload
torch.cuda.set_device(0)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
sys.path.insert(1, "/content/gdrive/My Drive/gentrl_v2")
import gentrl_v2

In [ ]:
os.chdir('/content/gdrive/My Drive/gentrl_v2')

In [ ]:
LATENT_SIZE = 8
LATENT_DESC = LATENT_SIZE * [('c', 10)]
FEATURE_DESC = [('c', 10), ('c', 10), ('d', 3), ('c', 10)]
BETA = 0.001
GAMMA = 0.25
# vae or cvae, changes between the VAE with structured prior and conditional VAE
VAE='vae'

# one experiment for each combination of data sampling method and VAE loss function:
# 009_random_cvae_rnn_50k for random data and CVAE
# 010_biased_cvae_rnn_50k for biased data and CVAE
# 011_random_vae_rnn_50k for random data and VAE with structured prior
# 012_biased_vae_rnn_50k for biased data and VAE with structured prior
EXPERIMENT_NAME = "012_biased_vae_rnn_50k"
BATCH_SIZE = 128
EPOCHS = 100

In [ ]:
enc = gentrl_v2.RNNEncoder(latent_size=LATENT_SIZE, bidirectional=True, hidden_size=128)
dec = gentrl_v2.RNNDecoder(latent_input_size=LATENT_SIZE)
model = gentrl_v2.GENTRL(enc, dec, LATENT_DESC, FEATURE_DESC, beta=BETA, gamma=GAMMA, vae=VAE, state_dim=LATENT_SIZE, action_dim=8)
model = model.cuda();

In [ ]:
reinit_epochs = [1, 2, 5, 25] # specify epochs to reset weights

In [ ]:
# change the path to load desired data - random or biased
md_train = gentrl_v2.MolecularDataset(
    {'path':'data/canonized/moses_biased_train_50k.csv',
     'smiles': 'SMILES',
     'prob': 1,
     'logP' : 'logP',
     'SA': 'SA',
     'nr_rings': 'nr_rings',
     'QED': 'QED',
    },
    props=['logP', 'SA', 'nr_rings', 'QED'])

md_val = gentrl_v2.MolecularDataset(
    {'path':'data/canonized/moses_biased_val_5k.csv',
     'smiles': 'SMILES',
     'prob': 1,
     'logP' : 'logP',
     'SA': 'SA',
     'nr_rings': 'nr_rings',
     'QED': 'QED',
    },
    props=['logP', 'SA', 'nr_rings', 'QED'])

In [ ]:
from torch.utils.data import DataLoader
train_loader = DataLoader(md_train, batch_size=BATCH_SIZE, shuffle=True, num_workers=0, drop_last=False)
val_loader = DataLoader(md_val, batch_size=BATCH_SIZE, shuffle=True, num_workers=0, drop_last=False)

In [ ]:
# run four times with corresponding random/biased data and vae/cvae setup to create 4 different latent spaces
model.train_as_vaelp(train_loader=train_loader,
                     train_name=EXPERIMENT_NAME,
                     val_loader=val_loader,
                     num_epochs=EPOCHS, lr=1e-3,
                     reinit_epochs=reinit_epochs,
                     start_epoch=restart_epoch)